In [1]:
## this is to preprocess the SBA data and then to create model and prediction

In [2]:
# imports
import pandas as pd
from google.cloud import storage
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [3]:
# create storage client to download data
storage_client = storage.Client()

In [4]:
# download the data
public_bucket = storage_client.bucket('sba_data_uoftcloudrj')
blob = public_bucket.blob('SBAnational.csv')

In [5]:
blob.download_to_filename('SBAnational.csv')

In [6]:
df = pd.read_csv('SBAnational.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df.head()

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,...,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,...,N,Y,NaN,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00"
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,...,N,N,NaN,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00"
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,28-Feb-97,1997,...,N,Y,NaN,30-Jun-97,"$35,000.00",$0.00,P I F,$0.00,"$35,000.00","$28,000.00"
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,28-Feb-97,1997,...,N,N,NaN,14-May-97,"$229,000.00",$0.00,P I F,$0.00,"$229,000.00","$229,000.00"


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899164 entries, 0 to 899163
Data columns (total 27 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   LoanNr_ChkDgt      899164 non-null  int64  
 1   Name               899150 non-null  object 
 2   City               899134 non-null  object 
 3   State              899150 non-null  object 
 4   Zip                899164 non-null  int64  
 5   Bank               897605 non-null  object 
 6   BankState          897598 non-null  object 
 7   NAICS              899164 non-null  int64  
 8   ApprovalDate       899164 non-null  object 
 9   ApprovalFY         899164 non-null  object 
 10  Term               899164 non-null  int64  
 11  NoEmp              899164 non-null  int64  
 12  NewExist           899028 non-null  float64
 13  CreateJob          899164 non-null  int64  
 14  RetainedJob        899164 non-null  int64  
 15  FranchiseCode      899164 non-null  int64  
 16  Ur

In [9]:
## with this model, our goal is to perform a classification to see whether the loan will be paid in full, or is likely to be charged off. in order to do this, we need to eliminate some of 

In [10]:
df.set_index('LoanNr_ChkDgt', inplace=True)

In [11]:
df.head()

,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
LoanNr_ChkDgt,,,,,,,,,,,,,,,,,,,,,
1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,84,...,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"
1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,60,...,N,Y,NaN,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00"
1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,180,...,N,N,NaN,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00"
1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,28-Feb-97,1997,60,...,N,Y,NaN,30-Jun-97,"$35,000.00",$0.00,P I F,$0.00,"$35,000.00","$28,000.00"
1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,28-Feb-97,1997,240,...,N,N,NaN,14-May-97,"$229,000.00",$0.00,P I F,$0.00,"$229,000.00","$229,000.00"


In [12]:
df.drop(columns=['City','State','BankState','ApprovalDate','DisbursementDate','BalanceGross','ChgOffPrinGr','ChgOffDate','Bank','Name','GrAppv'], inplace=True)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 899164 entries, 1000014003 to 9996003010
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Zip                899164 non-null  int64  
 1   NAICS              899164 non-null  int64  
 2   ApprovalFY         899164 non-null  object 
 3   Term               899164 non-null  int64  
 4   NoEmp              899164 non-null  int64  
 5   NewExist           899028 non-null  float64
 6   CreateJob          899164 non-null  int64  
 7   RetainedJob        899164 non-null  int64  
 8   FranchiseCode      899164 non-null  int64  
 9   UrbanRural         899164 non-null  int64  
 10  RevLineCr          894636 non-null  object 
 11  LowDoc             896582 non-null  object 
 12  DisbursementGross  899164 non-null  object 
 13  MIS_Status         897167 non-null  object 
 14  SBA_Appv           899164 non-null  object 
dtypes: float64(1), int64(8), object(6)
mem

In [14]:
df['NAICS'].value_counts()

0         201948
722110     27989
722211     19448
811111     14585
621210     14048
           ...  
312210         1
325180         1
339910         1
325192         1
336320         1
Name: NAICS, Length: 1312, dtype: int64

In [15]:
df.dropna(inplace=True)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889938 entries, 1000014003 to 9996003010
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Zip                889938 non-null  int64  
 1   NAICS              889938 non-null  int64  
 2   ApprovalFY         889938 non-null  object 
 3   Term               889938 non-null  int64  
 4   NoEmp              889938 non-null  int64  
 5   NewExist           889938 non-null  float64
 6   CreateJob          889938 non-null  int64  
 7   RetainedJob        889938 non-null  int64  
 8   FranchiseCode      889938 non-null  int64  
 9   UrbanRural         889938 non-null  int64  
 10  RevLineCr          889938 non-null  object 
 11  LowDoc             889938 non-null  object 
 12  DisbursementGross  889938 non-null  object 
 13  MIS_Status         889938 non-null  object 
 14  SBA_Appv           889938 non-null  object 
dtypes: float64(1), int64(8), object(6)
mem

In [17]:
df.head()

,Zip,NAICS,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,MIS_Status,SBA_Appv
LoanNr_ChkDgt,,,,,,,,,,,,,,,
1000014003,47711,451120,1997,84,4,2.0,0,0,1,0,N,Y,"$60,000.00",P I F,"$48,000.00"
1000024006,46526,722410,1997,60,2,2.0,0,0,1,0,N,Y,"$40,000.00",P I F,"$32,000.00"
1000034009,47401,621210,1997,180,7,1.0,0,0,1,0,N,N,"$287,000.00",P I F,"$215,250.00"
1000044001,74012,0,1997,60,2,1.0,0,0,1,0,N,Y,"$35,000.00",P I F,"$28,000.00"
1000054004,32801,0,1997,240,14,1.0,7,7,1,0,N,N,"$229,000.00",P I F,"$229,000.00"


In [18]:
df['NAICS'] = df['NAICS'].astype('str').apply(lambda x: x[:2])

In [19]:
df.head()

,Zip,NAICS,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,MIS_Status,SBA_Appv
LoanNr_ChkDgt,,,,,,,,,,,,,,,
1000014003,47711,45,1997,84,4,2.0,0,0,1,0,N,Y,"$60,000.00",P I F,"$48,000.00"
1000024006,46526,72,1997,60,2,2.0,0,0,1,0,N,Y,"$40,000.00",P I F,"$32,000.00"
1000034009,47401,62,1997,180,7,1.0,0,0,1,0,N,N,"$287,000.00",P I F,"$215,250.00"
1000044001,74012,0,1997,60,2,1.0,0,0,1,0,N,Y,"$35,000.00",P I F,"$28,000.00"
1000054004,32801,0,1997,240,14,1.0,7,7,1,0,N,N,"$229,000.00",P I F,"$229,000.00"


In [20]:
df['NAICS'].value_counts()

0     199482
44     84112
81     71965
54     67313
72     67189
23     65922
62     54856
42     48379
45     42033
33     37905
56     32253
48     20013
32     17772
71     14497
53     13495
31     11702
51     11254
52      9411
11      8900
61      6335
49      2186
21      1829
22       656
55       256
92       223
Name: NAICS, dtype: int64

In [21]:
df = df[df['NAICS'] != '0']

In [22]:
df = df[(df['RevLineCr'] == 'N') | (df['RevLineCr'] == 'Y')]
df['RevLineCr'] = np.where(df['RevLineCr'] == 'N', 0, 1)

In [23]:
df = df[(df['LowDoc'] == 'N') | (df['LowDoc'] == 'Y')]
df['LowDoc'] = np.where(df['LowDoc'] == 'N', 0, 1)

In [24]:
df['NAICS'] = df['NAICS'].astype('int64')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 459978 entries, 1000014003 to 9995613003
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Zip                459978 non-null  int64  
 1   NAICS              459978 non-null  int64  
 2   ApprovalFY         459978 non-null  object 
 3   Term               459978 non-null  int64  
 4   NoEmp              459978 non-null  int64  
 5   NewExist           459978 non-null  float64
 6   CreateJob          459978 non-null  int64  
 7   RetainedJob        459978 non-null  int64  
 8   FranchiseCode      459978 non-null  int64  
 9   UrbanRural         459978 non-null  int64  
 10  RevLineCr          459978 non-null  int64  
 11  LowDoc             459978 non-null  int64  
 12  DisbursementGross  459978 non-null  object 
 13  MIS_Status         459978 non-null  object 
 14  SBA_Appv           459978 non-null  object 
dtypes: float64(1), int64(10), object(4)
me

In [25]:
df['MIS_Status'].value_counts()

P I F     361408
CHGOFF     98570
Name: MIS_Status, dtype: int64

In [26]:
df['MIS_Status'] = np.where(df['MIS_Status'] == 'P I F', 0, 1)
df.head()

,Zip,NAICS,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,MIS_Status,SBA_Appv
LoanNr_ChkDgt,,,,,,,,,,,,,,,
1000014003,47711,45,1997,84,4,2.0,0,0,1,0,0,1,"$60,000.00",0,"$48,000.00"
1000024006,46526,72,1997,60,2,2.0,0,0,1,0,0,1,"$40,000.00",0,"$32,000.00"
1000034009,47401,62,1997,180,7,1.0,0,0,1,0,0,0,"$287,000.00",0,"$215,250.00"
1000084002,6062,33,1997,120,19,1.0,0,0,1,0,0,0,"$517,000.00",0,"$387,750.00"
1000094005,34491,81,1997,84,1,2.0,0,0,1,0,0,1,"$45,000.00",0,"$36,000.00"


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 459978 entries, 1000014003 to 9995613003
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Zip                459978 non-null  int64  
 1   NAICS              459978 non-null  int64  
 2   ApprovalFY         459978 non-null  object 
 3   Term               459978 non-null  int64  
 4   NoEmp              459978 non-null  int64  
 5   NewExist           459978 non-null  float64
 6   CreateJob          459978 non-null  int64  
 7   RetainedJob        459978 non-null  int64  
 8   FranchiseCode      459978 non-null  int64  
 9   UrbanRural         459978 non-null  int64  
 10  RevLineCr          459978 non-null  int64  
 11  LowDoc             459978 non-null  int64  
 12  DisbursementGross  459978 non-null  object 
 13  MIS_Status         459978 non-null  int64  
 14  SBA_Appv           459978 non-null  object 
dtypes: float64(1), int64(11), object(3)
me

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 459978 entries, 1000014003 to 9995613003
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Zip                459978 non-null  int64  
 1   NAICS              459978 non-null  int64  
 2   ApprovalFY         459978 non-null  object 
 3   Term               459978 non-null  int64  
 4   NoEmp              459978 non-null  int64  
 5   NewExist           459978 non-null  float64
 6   CreateJob          459978 non-null  int64  
 7   RetainedJob        459978 non-null  int64  
 8   FranchiseCode      459978 non-null  int64  
 9   UrbanRural         459978 non-null  int64  
 10  RevLineCr          459978 non-null  int64  
 11  LowDoc             459978 non-null  int64  
 12  DisbursementGross  459978 non-null  object 
 13  MIS_Status         459978 non-null  int64  
 14  SBA_Appv           459978 non-null  object 
dtypes: float64(1), int64(11), object(3)
me

In [29]:
df[['DisbursementGross','SBA_Appv']] = df[['DisbursementGross','SBA_Appv']].applymap(lambda x: x.strip().replace('$','').replace(',',''))

In [30]:
df.head()

,Zip,NAICS,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,MIS_Status,SBA_Appv
LoanNr_ChkDgt,,,,,,,,,,,,,,,
1000014003,47711,45,1997,84,4,2.0,0,0,1,0,0,1,60000.00,0,48000.00
1000024006,46526,72,1997,60,2,2.0,0,0,1,0,0,1,40000.00,0,32000.00
1000034009,47401,62,1997,180,7,1.0,0,0,1,0,0,0,287000.00,0,215250.00
1000084002,6062,33,1997,120,19,1.0,0,0,1,0,0,0,517000.00,0,387750.00
1000094005,34491,81,1997,84,1,2.0,0,0,1,0,0,1,45000.00,0,36000.00


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 459978 entries, 1000014003 to 9995613003
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Zip                459978 non-null  int64  
 1   NAICS              459978 non-null  int64  
 2   ApprovalFY         459978 non-null  object 
 3   Term               459978 non-null  int64  
 4   NoEmp              459978 non-null  int64  
 5   NewExist           459978 non-null  float64
 6   CreateJob          459978 non-null  int64  
 7   RetainedJob        459978 non-null  int64  
 8   FranchiseCode      459978 non-null  int64  
 9   UrbanRural         459978 non-null  int64  
 10  RevLineCr          459978 non-null  int64  
 11  LowDoc             459978 non-null  int64  
 12  DisbursementGross  459978 non-null  object 
 13  MIS_Status         459978 non-null  int64  
 14  SBA_Appv           459978 non-null  object 
dtypes: float64(1), int64(11), object(3)
me

In [32]:
df['ApprovalFY'].unique()

array([1997, 2006, 1998, 1999, 2000, 2001, 2003, 2004, 2005, 1984, 2007,
       1987, 2008, 1988, 2009, 1989, 1990, 2010, 1991, 2011, 1992, 2002,
       2012, 1993, 2013, 1994, 2014, '2004', '1994', '1995', '2005',
       '1996', 1996], dtype=object)

In [33]:
def cleaned(x):
    if isinstance(x, str):
        return x.replace('A','')
    return x

df['ApprovalFY'] = df['ApprovalFY'].apply(cleaned).astype('int64')

In [34]:
df['ApprovalFY'].unique()

array([1997, 2006, 1998, 1999, 2000, 2001, 2003, 2004, 2005, 1984, 2007,
       1987, 2008, 1988, 2009, 1989, 1990, 2010, 1991, 2011, 1992, 2002,
       2012, 1993, 2013, 1994, 2014, 1995, 1996])

In [35]:
df.head()

,Zip,NAICS,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,MIS_Status,SBA_Appv
LoanNr_ChkDgt,,,,,,,,,,,,,,,
1000014003,47711,45,1997,84,4,2.0,0,0,1,0,0,1,60000.00,0,48000.00
1000024006,46526,72,1997,60,2,2.0,0,0,1,0,0,1,40000.00,0,32000.00
1000034009,47401,62,1997,180,7,1.0,0,0,1,0,0,0,287000.00,0,215250.00
1000084002,6062,33,1997,120,19,1.0,0,0,1,0,0,0,517000.00,0,387750.00
1000094005,34491,81,1997,84,1,2.0,0,0,1,0,0,1,45000.00,0,36000.00


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 459978 entries, 1000014003 to 9995613003
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Zip                459978 non-null  int64  
 1   NAICS              459978 non-null  int64  
 2   ApprovalFY         459978 non-null  int64  
 3   Term               459978 non-null  int64  
 4   NoEmp              459978 non-null  int64  
 5   NewExist           459978 non-null  float64
 6   CreateJob          459978 non-null  int64  
 7   RetainedJob        459978 non-null  int64  
 8   FranchiseCode      459978 non-null  int64  
 9   UrbanRural         459978 non-null  int64  
 10  RevLineCr          459978 non-null  int64  
 11  LowDoc             459978 non-null  int64  
 12  DisbursementGross  459978 non-null  object 
 13  MIS_Status         459978 non-null  int64  
 14  SBA_Appv           459978 non-null  object 
dtypes: float64(1), int64(12), object(2)
me

In [37]:
df['DisbursementGross'] = df['DisbursementGross'].astype('float')

In [38]:
df['SBA_Appv'] = df['SBA_Appv'].astype('float')

In [41]:
df['FranchiseCode'].value_counts()

1        265149
0        180462
78760       828
68020       310
21780       183
          ...  
26554         1
47520         1
85150         1
9651          1
42475         1
Name: FranchiseCode, Length: 1696, dtype: int64

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 459978 entries, 1000014003 to 9995613003
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Zip                459978 non-null  int64  
 1   NAICS              459978 non-null  int64  
 2   ApprovalFY         459978 non-null  int64  
 3   Term               459978 non-null  int64  
 4   NoEmp              459978 non-null  int64  
 5   NewExist           459978 non-null  float64
 6   CreateJob          459978 non-null  int64  
 7   RetainedJob        459978 non-null  int64  
 8   FranchiseCode      459978 non-null  int64  
 9   UrbanRural         459978 non-null  int64  
 10  RevLineCr          459978 non-null  int64  
 11  LowDoc             459978 non-null  int64  
 12  DisbursementGross  459978 non-null  float64
 13  MIS_Status         459978 non-null  int64  
 14  SBA_Appv           459978 non-null  float64
dtypes: float64(3), int64(12)
memory usage:

In [40]:
df = df.sample(frac=1)

In [42]:
df_train = df[:413000]
df_new = df[413000:]

In [43]:
df_new.drop(columns=['MIS_Status'],inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [44]:
df_train.to_csv('training_data.csv')
df_new.to_csv('new_data.csv')

In [45]:
storage_client = storage.Client()
bucket = storage_client.bucket('sba_data_uoftcloudrj')
blob = bucket.blob('training_data.csv')
blob.upload_from_filename('training_data.csv')
blob = bucket.blob('new_data.csv')
blob.upload_from_filename('new_data.csv')